In [7]:
from typing import Annotated
from dotenv import load_dotenv

load_dotenv()

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [4]:
from langchain.chat_models import init_chat_model

In [ ]:
llm = init_chat_model("google_genai:gemini-2.0-flash")
# llm.invoke("who was the first person to walk on moon?")

In [8]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [9]:
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
# message = {"role": "user", "content": "What is the latest price of MSFT stock?"}
response = graph.invoke({"messages":[message]})

response["messages"]

[HumanMessage(content='Who walked on the moon for the first time? Print only the name', additional_kwargs={}, response_metadata={}, id='ef38e9c8-ea58-4b36-9817-2d5fb025c0de'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--bd6ee694-d706-4495-a9eb-d625857c05f5-0', usage_metadata={'input_tokens': 14, 'output_tokens': 3, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})]

In [10]:
# to get messages one by one line from input to chatbot
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in {"quit","exit"}:
        break
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)

Bot: The telegram was not invented by a single person, but rather developed over time by several inventors. However, **Samuel Morse** is most famously associated with the invention of the telegraph system that became widely adopted.

Here's a breakdown:

*   **Early Concepts:** The idea of electrical telegraphy existed before Morse. Various inventors experimented with transmitting messages using electricity.
*   **Samuel Morse (and Alfred Vail):** Morse developed the single-wire telegraph system, and more importantly, he and his associate Alfred Vail created the Morse code, a system of dots and dashes representing letters and numbers. This code was essential for transmitting messages efficiently. Morse secured a patent for his telegraph in 1840.
*   **Other Contributors:** Many other inventors contributed to the development of telegraph technology, including:

    *   **Claude Chappe:** Developed a semaphore telegraph system (using visual signals) in France before electrical telegraphy

KeyboardInterrupt: Interrupted by user